# 슈퍼바이저 멀티 에이전트 
- 구글 제미나이 사용
https://medium.com/@dharamai2024/build-a-multi-agent-system-with-langgraph-5b938e0104b5

Gemini API키는 다음 사이트에서 [Google AI 스튜디오](https://aistudio.google.com/app/apikey?hl=ko) 등록합니다.

<img src="https://miro.medium.com/v2/resize:fit:786/format:webp/1*MLHRiyXv7py7vT--wupJeA.png" />

## 환경 설정

In [1]:
!python --version

Python 3.13.4


In [2]:
%pip install -qU python-dotenv langchain pypdf langchain-community langgraph langchain-tavily langchain-google-genai langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
# 환경 변수 확인
from dotenv import load_dotenv
import os
load_dotenv(override=True)

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("GOOGLE_API_KEY")
_set_env("TAVILY_API_KEY")

print(os.environ.get("OPENAI_API_KEY")[:10])
print(os.environ.get("GOOGLE_API_KEY")[:10])
print(os.environ.get("TAVILY_API_KEY")[:10])

sk-proj-8x
AIzaSyB_bx
tvly-dev-G


In [4]:

import os
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from typing import Annotated, TypedDict, Literal, Sequence
from pydantic import BaseModel, Field
from langgraph.graph import START, StateGraph, MessagesState, add_messages, END
from langgraph.graph.message import add_messages
from IPython.display import Image, display
from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage, SystemMessage
from langgraph.prebuilt import ToolNode, tools_condition
from langchain.schema import HumanMessage
from langchain.tools import Tool
from langchain_core.tools import tool
from langchain_core.messages import ToolMessage
from langchain_core.tools import InjectedToolCallId, tool
from langgraph.types import Command, interrupt
from langchain_core.output_parsers import StrOutputParser


In [5]:
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
# from langchain_google_genai import ChatGoogleGenerativeAI

# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
# llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", 
#                              temperature=0)

In [6]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",  # 가격이 저렴
)
llm = ChatOpenAI(model_name="gpt-4o-mini")

## 도구 만들기

LangGraph를 사용하는 이 다중 에이전트 시스템 은 7명의 전문 에이전트 로 구성되며 , 각 에이전트는 고유한 책임과 툴셋을 가지고 있습니다. 핵심적으로, 슈퍼바이저 에이전트는 쿼리 유형에 따라 가장 적합한 에이전트에게 작업을 동적으로 라우팅하여 전체 워크플로를 조율합니다.

저희 에이전트 팀에 대한 개요는 다음과 같습니다.

- **Supervisor Agent**는 중앙 조정자 역할을 합니다. 사용자 입력을 받아 적절한 전문 에이전트에게 작업을 위임합니다.
- **Enhancer Agent**는 다른 에이전트에게 전달되기 전에 사용자의 쿼리를 세부화하거나 향상시킵니다(예: 명확성이나 맥락 추가).
- **Validator Agent**는 사용자의 질문과 최종 응답을 모두 검증하여 정확성, 완전성, 안전성을 보장합니다.
- **Researcher Agent**는 검색 엔진 도구(예: Tavily)를 사용하여 실시간 웹 조사를 수행하여 최신 정보를 검색합니다.
- **PDF_RAG Agent**는 PDF 검색기를 사용하여 PDF 컬렉션에서 관련 콘텐츠를 검색하고 RAG(검색 증강 생성) 기술을 적용합니다.
- **Movie_RAG Agent**는 CSV 기반 검색기를 사용하여 영화 데이터 세트와 관련된 질문에 답합니다.
- **Math Expert Agent**는 Wolfram이나 LangChain의 내장된 수학 기능과 같은 계산 도구를 사용하여 수학 문제를 해결합니다.

## Research Agent Tool:

In [7]:
from langchain_tavily import TavilySearch

web_search_tool = TavilySearch(max_results=2)

## PDF Agnet Tool

In [8]:
from langchain_community.document_loaders import PyPDFLoader, CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_pdf_documents(file_path):
  documents = []
  for file in os.listdir(file_path):
    if file.endswith(".pdf"):
      loader = PyPDFLoader(os.path.join(file_path, file))
      documents.extend(loader.load())
  return documents

# upload and create documents
documents = load_pdf_documents("./docs")
print(f"{len(documents)} documents loaded")

28 documents loaded


In [9]:
text_splitter = RecursiveCharacterTextSplitter(
     chunk_size=1000,
     chunk_overlap=200,
     length_function=len
 )

splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")

Split the documents into 71 chunks.


In [10]:
!pip install -qU "langchain-chroma>=0.1.2"

In [28]:
 from langchain_chroma import Chroma

 collection_name = "my_collection"
 vectorstore = Chroma.from_documents(
     collection_name=collection_name,
     documents=splits,
     embedding=embeddings
 )


TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates